In [17]:
import sys
import pandas as pd
import numpy as np

# loading train.csv file, encoding as big 5
data = pd.read_csv('date/train.csv', encoding='big5')
# show first 10rows
data.head(5)

,???,????,????.1,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,???,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,???,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,???,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,???,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,???,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


preprocessing

In [18]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
# convert data  to 2 dimesion
raw_data = data.to_numpy()
print(raw_data)

[['14' '14' '14' ... '15' '15' '15']
 ['1.8' '1.8' '1.8' ... '1.8' '1.8' '1.8']
 ['0.51' '0.41' '0.39' ... '0.35' '0.36' '0.32']
 ...
 ['36' '55' '72' ... '118' '100' '105']
 ['1.9' '2.4' '1.9' ... '1.5' '2' '2']
 ['0.7' '0.8' '1.8' ... '1.6' '1.8' '2']]


extract features

In [ ]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 20*24])
    for day in range(20):
        sample[:, day * 24 : (day+1)*24] = raw_data[18*(20*month+day): 18*(20*month+day+1), :]
    month_data[month] = sample
# print(month_data)

x = np.empty([12 * 471, 18 * 9], dtype=float)
y = np.empty([12 * 471, 1], dtype=float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month*471 + day*24 + hour, :] = month_data[month][:, day*24 + hour: day*24 + hour + 9].reshape(1, -1)  # vector
            y[month*471 + day*24 + hour, 0] = month_data[month][9, day*24 + hour + 9] # value
print(x)
print(y)

normalize



In [ ]:
mean_x = np.mean(x, axis=0)
std_x = np.std(x, axis=0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
print(x)
